In [1]:
import sys
import argparse
import warnings
from pathlib import Path
from scripts.processing import Processor
from scripts.folder_structure import FolderStructure
from scripts.analyze import load_pyseer_hits
from scripts.analyze import AnalyzeResults
from scripts.figures import get_k_loci, FIGURE4_PANELA, FIGURE4_PANELB


%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

# workflow IO
input_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT"
output_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-03-26_KLEBDATA_LIGHT_GWAS"

In [2]:
# folder structure
folder_manager = FolderStructure(input_dir, output_dir)
structure = folder_manager.get_paths_dict()
params = folder_manager.get_params_dict()

# aggregate results
processor = Processor(structure, params)
processor.aggregate_pyseer_hits()

# load
pyseer_hits = load_pyseer_hits(structure, params)

NameError: name 'proteins_functions_table' is not defined

In [68]:
analyze = AnalyzeResults(structure, params)

analyze.per_locus_analysis(run=True)
analyze.aggregate_data(run=False)
analyze.combine_predictions_and_enzymes(run=False)

Precision and recall plots per capsule... 
/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-02-12_KLEBDATA_LIGHT_GWAS/4_ANALYZE/1_PER_LOCUS/KL1/elastic_net/KL1_elastic_net.pdf


In [65]:
# FIGURE4_PANELA(structure, params, merge_expr=True, target_fig_height=1.4, target_fig_width=7.5, \
#                labels_fontsize=4, cells_fontsize=3.5, margin=0.3, header_space=0.1)

# FIGURE4_PANELB(structure, params, min_cov=0.5, min_pident=0, max_evalue=1e-3, use_coverage=True)